In [2]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [3]:
URL = 'https://www.lequipe.fr/Football/ligue-1/saison-2020-2021/page-calendrier-resultats'
page = requests.get(URL)
soup = BeautifulSoup(page.content, 'html.parser')
results = soup.find_all('select', class_='SelectNav__select')

In [4]:
list_url_journey = []
for result in results:
    liens_journey = result.find_all('option')
    for liens in liens_journey:
        list_url_journey.append(liens.get('value'))

In [14]:
row_list = []
for url_journey in list_url_journey:
    journey = pd.read_json(url_journey, lines=True)
    for match in journey['items'][0]:
        try:
            if len(match['items']) == 1:
                match_dict={}
                match_dict['place'] = match['items'][0]['event']['lieu']['ville']
                match_dict['date'] = match['items'][0]['event']['date']
                match_dict['dom_name'] = match['items'][0]['event']['specifics']['domicile']['equipe']['nom']
                match_dict['ext_name'] = match['items'][0]['event']['specifics']['exterieur']['equipe']['nom']
                match_dict['but_dom'] = match['items'][0]['event']['specifics']['score']['domicile']
                match_dict['but_ext'] = match['items'][0]['event']['specifics']['score']['exterieur']
                match_dict['winner'] = match['items'][0]['event']['specifics']['vainqueur']
                match_dict['lien'] = match['items'][0]['event']['lien_web']
                row_list.append(match_dict)
            elif len(match['items']) > 1:
                for m in range(len(match['items'])):
                    match_dict={}
                    match_dict['place'] = match['items'][m]['event']['lieu']['ville']
                    match_dict['date'] = match['items'][m]['event']['date']
                    match_dict['dom_name'] = match['items'][m]['event']['specifics']['domicile']['equipe']['nom']
                    match_dict['ext_name'] = match['items'][m]['event']['specifics']['exterieur']['equipe']['nom']
                    match_dict['but_dom'] = match['items'][m]['event']['specifics']['score']['domicile']
                    match_dict['but_ext'] = match['items'][m]['event']['specifics']['score']['exterieur']
                    match_dict['winner'] = match['items'][m]['event']['specifics']['vainqueur']
                    match_dict['lien'] = match['items'][m]['event']['lien_web']
                    row_list.append(match_dict)
        except:
            pass

data = pd.DataFrame(row_list)

In [15]:
data.shape

(380, 8)

In [16]:
data.head()

,place,date,dom_name,ext_name,but_dom,but_ext,winner,lien
0,Bordeaux,2020-08-21T19:00:00+0200,Bordeaux,Nantes,0,0,ex-aequo,https://www.lequipe.fr/Football/match-direct/l...
1,Dijon,2020-08-22T17:00:00+0200,Dijon,Angers,0,1,exterieur,https://www.lequipe.fr/Football/match-direct/l...
2,Lille,2020-08-22T21:00:00+0200,Lille,Rennes,1,1,ex-aequo,https://www.lequipe.fr/Football/match-direct/l...
3,Monaco,2020-08-23T13:00:00+0200,Monaco,Reims,2,2,ex-aequo,https://www.lequipe.fr/Football/match-direct/l...
4,Lorient,2020-08-23T15:00:00+0200,Lorient,Strasbourg,3,1,domicile,https://www.lequipe.fr/Football/match-direct/l...


In [20]:
df_matchs = data[['place', 'date']]
df_matchs

,place,date
0,Bordeaux,2020-08-21T19:00:00+0200
1,Dijon,2020-08-22T17:00:00+0200
2,Lille,2020-08-22T21:00:00+0200
3,Monaco,2020-08-23T13:00:00+0200
4,Lorient,2020-08-23T15:00:00+0200
...,...,...
375,Rennes,2021-05-23T21:00:00+0200
376,Nantes,2021-05-23T21:00:00+0200
377,Saint-Étienne,2021-05-23T21:00:00+0200
378,Reims,2021-05-23T21:00:00+0200


In [19]:
list_liens_matchs = data['lien'].tolist()
list_liens_matchs

['https://www.lequipe.fr/Football/match-direct/ligue-1/2020-2021/bordeaux-nantes-live/477156',
 'https://www.lequipe.fr/Football/match-direct/ligue-1/2020-2021/dijon-angers-live/477160',
 'https://www.lequipe.fr/Football/match-direct/ligue-1/2020-2021/lille-rennes-live/477162',
 'https://www.lequipe.fr/Football/match-direct/ligue-1/2020-2021/monaco-reims-live/477164',
 'https://www.lequipe.fr/Football/match-direct/ligue-1/2020-2021/lorient-strasbourg-live/477157',
 'https://www.lequipe.fr/Football/match-direct/ligue-1/2020-2021/nimes-brest-live/477158',
 'https://www.lequipe.fr/Football/match-direct/ligue-1/2020-2021/nice-lens-live/477159',
 'https://www.lequipe.fr/Football/match-direct/ligue-1/2020-2021/montpellier-ol-live/477155',
 'https://www.lequipe.fr/Football/match-direct/ligue-1/2020-2021/psg-metz-live/477163',
 'https://www.lequipe.fr/Football/match-direct/ligue-1/2020-2021/om-asse-live/477161',
 'https://www.lequipe.fr/Football/match-direct/ligue-1/2020-2021/ol-dijon-live/477